In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('../data/cleaned_cuisines.csv')
data.head()

X = data.iloc[:, 2:]
# X = X.apply(LabelEncoder().fit_transform)

y = data[['cuisine']]
# y = y.apply(LabelEncoder().fit_transform)

/Users/chuisi/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)


In [4]:
model = SVC(kernel='linear', C=10, probability=True, random_state=0)
model.fit(X_train, y_train.values.ravel())


SVC(C=10, kernel='linear', probability=True, random_state=0)

In [6]:
y_pred = model.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

     chinese       0.73      0.71      0.72       243
      indian       0.82      0.88      0.85       249
    japanese       0.76      0.70      0.73       238
      korean       0.84      0.76      0.80       244
        thai       0.72      0.83      0.77       225

    accuracy                           0.77      1199
   macro avg       0.77      0.77      0.77      1199
weighted avg       0.78      0.77      0.77      1199



In [7]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [('float_input', FloatTensorType([None, 380]))]
options = {id(model): {'nocl': True, 'zipmap': False}}

In [8]:
onx = convert_sklearn(model, initial_types=initial_type, options=options)
with open("./model.onnx", "wb") as f:
    f.write(onx.SerializeToString())